In [3]:
# 📚 Basic libraries
import pandas as pd
import numpy as np 

# 🤖 Machine Learning
# models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error


# Project libraries
from utils.file_handler import *
from utils.plots import *
from utils import eda
from utils.linear_regression import linear_regression_test
from utils.common_viz import model_validation
from utils.common_viz import corr_heatmap

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
import warnings
warnings.filterwarnings('ignore') # ignore warnings

# Logging
import logging
# setup logging level
logging.getLogger().setLevel(logging.DEBUG)

## Feature engineering
Import results from feature engineering performed in feature_engineering.ipynb

In [6]:
df_local_path = "dataframes/df_featuretuned_linear_reg.pkl"
df = read_df_pickle(df_local_path)
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,zipcode_cluster,zipcode_encoded,yr_sold,building_age,yrs_since_renovation,bath_bed_ratio,sqft_per_room,log_sqft_lot,log_sqft_lot15,log_sqft_above,log_sqft_basement,log_sqft_living,sale_count,multiple_sales,yrs_since_previous_sale
2496,2014-09-16,280000.0,6,3.00,2400,9373,2.0,0,0,3,7,2400,0,1991,0,47.3262,-122.214,2060,7316,2,1,2014,23,0,0.500000,266.666667,9.145695,8.897956,7.783641,0.000000,7.783641,2,1,0.0
2497,2015-04-22,300000.0,6,3.00,2400,9373,2.0,0,0,3,7,2400,0,1991,0,47.3262,-122.214,2060,7316,2,1,2015,24,0,0.500000,266.666667,9.145695,8.897956,7.783641,0.000000,7.783641,2,1,1.0
6735,2014-05-08,647500.0,4,1.75,2060,26036,1.0,0,0,4,8,1160,900,1947,0,47.4444,-122.351,2590,21891,1,63,2014,67,0,0.437500,358.260870,10.167274,9.993877,7.057037,6.803505,7.630947,1,0,0.0
8411,2014-08-11,400000.0,3,1.00,1460,43000,1.0,0,0,3,7,1460,0,1952,0,47.4434,-122.347,2250,20023,1,63,2014,62,0,0.333333,365.000000,10.668979,9.904687,7.286876,0.000000,7.286876,1,0,0.0
8809,2015-04-01,235000.0,3,1.00,1430,7599,1.5,0,0,4,6,1010,420,1930,0,47.4783,-122.265,1290,10320,1,64,2015,85,0,0.333333,357.500000,8.935904,9.241936,6.918695,6.042633,7.266129,1,0,0.0


In [7]:
target_variable = 'price'

continuous_features = [['bath_bed_ratio', 'lat', 'log_sqft_above', 'log_sqft_basement', 'log_sqft_living', 'log_sqft_lot', 'log_sqft_lot15', 'long', 'sqft_above', 'sqft_basement', 'sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_lot15', 'sqft_per_room']]

descrete_features = [['bathrooms', 'bedrooms', 'building_age', 'floors', 'multiple_sales', 'sale_count', 'view', 'yr_built', 'yr_renovated', 'yr_sold', 'yrs_since_previous_sale', 'yrs_since_renovation', 'zipcode_cluster', 'zipcode_encoded']]

ordinal_encoded_categorical_features = [['condition', 'grade', 'waterfront']]

all_numeric_features = continuous_features + descrete_features

all_features = all_numeric_features + ordinal_encoded_categorical_features

## Linear Regression Hyperparameters tuning

In [ ]:
logging.getLogger().setLevel(logging.INFO)

# Define different subsets of features
feature_subsets = {
    "continuous": continuous_features,
    "discrete": descrete_features,
    "categorical": ordinal_encoded_categorical_features,
    "all_numeric": all_numeric_features
}

# Define test sizes to experiment with
test_sizes = [0.2, 0.3, 0.4]

# Define different random_state values for variability
random_states = [15, 42, 100]

# Create dictionary to store test results
dict_model_test = {}

# Loop through different feature subsets, test sizes, and random states
for subset_name, feature_list in feature_subsets.items():
    for test_size in test_sizes:
        for random_state in random_states:
            # X/y split
            X = df[feature_list]
            y = df[target_variable]

            # Run linear regression test
            key = f"LR_{subset_name}_ts{test_size}_rs{random_state}"
            dict_model_test[key] = linear_regression_test(X, y, test_size, random_state)

df_model_test = pd.DataFrame(dict_model_test).T

# Apply formatting to ensure all values are displayed with 4 decimal places
columns_to_format = ['R2', 'MAE', 'RMSE', 'MSE']
df_model_test[columns_to_format] = df_model_test[columns_to_format].map(lambda x: f"{x:.4f}")

df_model_test.sort_values('R2', ascending=False, inplace=True)

df_model_test

,test_size,random_state,R2,MAE,RMSE,MSE
LR_all_numeric_ts0.2_rs15,0.2,15.0,0.7128,124402.4665,203006.7754,41211750865.0296
LR_all_numeric_ts0.3_rs15,0.3,15.0,0.7027,127105.1038,207959.6066,43247197966.1649
LR_all_numeric_ts0.4_rs15,0.4,15.0,0.6937,128083.5874,208299.5182,43388689297.3495
LR_all_numeric_ts0.4_rs100,0.4,100.0,0.6922,129307.3639,215034.9612,46240034517.3156
LR_all_numeric_ts0.3_rs100,0.3,100.0,0.6848,130815.2597,214304.7771,45926537508.0975
LR_all_numeric_ts0.4_rs42,0.4,42.0,0.6846,127676.4889,198895.7666,39559525982.6036
LR_all_numeric_ts0.3_rs42,0.3,42.0,0.6796,128027.7197,198387.5351,39357614087.2996
LR_all_numeric_ts0.2_rs100,0.2,100.0,0.6761,129810.0785,209166.0131,43750421022.2316
LR_all_numeric_ts0.2_rs42,0.2,42.0,0.6627,128602.0373,200743.4795,40297944556.7745
LR_continuous_ts0.2_rs15,0.2,15.0,0.6572,137242.1804,221760.4922,49177715896.3756


## Linear Regression 
- All numeric features
- test size 20%
- random state 15

In [ ]:
# Test/Train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

logging.debug(f'70% for training data: {len(X_train)}.')
logging.debug(f'30% for test data: {len(X_test)}.')

# Train the model
final_linear_regression_model = LinearRegression()
final_linear_regression_model.fit(X_train,y_train)

# Make predictions on the test dataset
predictions = final_linear_regression_model.predict(X_test)
r2_3 = r2_score(y_test, predictions)
MAE_3 = mean_absolute_error(y_test, predictions)
RMSE_3 = root_mean_squared_error(y_test, predictions)
MSE_3 = mean_squared_error(y_test, predictions)

#Printing the results
print(f"R2 = {round(r2_3, 4)}")
print(f"MAE = {round(MAE_3, 4)}")
print(f"RMSE = {round(RMSE_3, 4)}")
print(f"MSE =  {round(MSE_3, 4)}")

R2 = 0.7128
MAE = 124402.4665
RMSE = 203006.7754
MSE =  41211750865.0296


In [ ]:
# Save model to a file
save_model_pickle(model=final_linear_regression_model, filename='final_linear_regression_model')


INFO:root:Model file pickle is updated: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/mini_projects/mini-project-ironkaggle/models/final_linear_regression_model.json
